# 01 - Scraping des Avis Stores (Pas à Pas)

Dans ce notebook, nous allons construire notre scraper étape par étape. L'objectif est de comprendre comment récupérer les données brutes avant de les transformer en un format utilisable.

## 1. Préparation
Installation des librairies nécessaires.

In [ ]:
import pandas as pd
import requests
from google_play_scraper import reviews, Sort
from datetime import datetime, timedelta
print("Tout est OK !")

## 2. Google Play Store

La librairie `google-play-scraper` facilite grandement le travail. Elle simule le comportement du navigateur.

### Étape 2.1 : Récupérer un échantillon
Commençons par récupérer simplement les 3 derniers avis pour voir à quoi ressemble la donnée brute.

In [ ]:
app_id = 'ai.mistral.chat'

result, continuation_token = reviews(
    app_id,
    lang='fr', 
    country='fr', 
    sort=Sort.NEWEST, 
    count=3 
)

# Affichons le premier avis brut
first_review = result[1]
print(f"Date: {first_review['at']}")
print(f"Note: {first_review['score']}")
print(f"Texte: {first_review['content']}")
print("\nDonnées brutes complètes :")
print(first_review)

### Étape 2.2 : Structurer les données
Nous voyons que les champs intéressants sont `at` (date), `score` (note), `content` (commentaire) et `reviewCreatedVersion` (version de l'app).

### Étape 2.3 : La logique de filtrage (Boucle)
Avant de créer une fonction, essayons de traiter manuellement les résultats que nous venons d'obtenir. Nous allons boucler sur la liste `result` et extraire uniquement ce qui nous intéresse.

In [ ]:
# Imaginons que 'result' contient nos avis bruts de l'étape précédente
cleaned_data = []

# Définissons une période cible (ex: 30 derniers jours)
end = datetime.now().date()

start = end - timedelta(days=30)
print(f"Filtrage entre {start} et {end}...")

for r in result:
    review_date = r['at'].date()
    
    # On garde seulement si c'est dans la période
    if start <= review_date <= end:
        cleaned_data.append({
            'source': 'Google Play',
            'date': review_date,
            'rating': r['score'],
            'text': r['content'],
            'version': r.get('reviewCreatedVersion', 'N/A')
        })

print(f"{len(cleaned_data)} avis conservés après filtrage.")
pd.DataFrame(cleaned_data).head()

### Étape 2.4 : Création de la fonction finale
Maintenant que la logique est validée, on "emballe" tout ça dans une fonction réutilisable.

In [ ]:
def fetch_google_play(app_id, start_date, end_date):
    print(f"🔍 Google Play : Récupération pour {app_id}...")
    
    # On demande 2000 avis pour être large
    all_reviews, _ = reviews(
        app_id,
        lang='fr', 
        country='fr', 
        sort=Sort.NEWEST, 
        count=2000
    )
    
    cleaned_data = []
    for r in all_reviews:
        review_date = r['at'].date()
        
        # On garde uniquement les avis dans notre fenêtre de temps
        if start_date <= review_date <= end_date:
            cleaned_data.append({
                'source': 'Google Play',
                'date': review_date,
                'rating': r['score'],
                'text': r['content'],
                'version': r.get('reviewCreatedVersion', 'N/A')
            })
        elif review_date < start_date:
            # Comme les avis sont triés par date décroissante, on peut arrêter dès qu'on sort de la fenêtre
            break
            
    return pd.DataFrame(cleaned_data)

# Test final de la fonction
df_gp = fetch_google_play('ai.mistral.chat', start, end)
print(df_gp.shape)
df_gp.head(15)


## 3. Apple App Store

Pour Apple, c'est plus "artisanal". Il n'y a pas de librairie officielle simple, mais un flux RSS public est accessible.

### Étape 3.1 : Comprendre le flux RSS
L'URL ressemble à ceci : `https://itunes.apple.com/fr/rss/customerreviews/page=1/id={APP_ID}/sortby=mostrecent/json`.
Essayons d'appeler la première page.

In [ ]:
apple_id = '6740410176'
url = f"https://itunes.apple.com/fr/rss/customerreviews/page=1/id={apple_id}/sortby=mostrecent/json"

response = requests.get(url)
print(f"Status Code: {response.status_code}")

data = response.json()
entries = data['feed']['entry']
print(f"Nombre d'entrées récupérées : {len(entries)}")
print(entries)

### Étape 3.2 : Analyser une entrée
Attention : la première entrée `entries[0]` est souvent les métadonnées de l'application, pas un avis utilisateur ! Regardons la deuxième entrée pour comprendre où sont cachées les infos.

In [ ]:
if len(entries) > 1:
    review = entries[1]
    # Les champs sont bizarres, c'est du format RSS converti en JSON
    print(f"Note: {review['im:rating']['label']}")
    print(f"Texte: {review['content']['label']}")
    print(f"Date: {review['updated']['label']}")

### Étape 3.3 : Comprendre la pagination
Un appel ne renvoie que 50 avis maximum. Pour en avoir plus, il faut changer le numéro de page dans l'URL (`page=1`, `page=2`, etc.).

In [ ]:
# Exemple : Construction de l'URL pour la page 2
page = 2
url_page_2 = f"https://itunes.apple.com/fr/rss/customerreviews/page={page}/id={apple_id}/sortby=mostrecent/json"
print(f"URL Page 2 : {url_page_2}")

# C'est cette URL qu'il faudra appeler dans une boucle.

### Étape 3.4 : Création de la fonction finale (avec Pagination)
Nous allons créer une boucle `for page in range(1, 11)` pour récupérer jusqu'à 10 pages d'avis.

In [ ]:
def fetch_app_store(app_id, start_date, end_date):
    print(f"🍏 App Store : Récupération pour {app_id}...")
    all_reviews = []
    
    # On parcourt les 10 premières pages (500 avis max)
    for page in range(1, 11):
        url = f"https://itunes.apple.com/fr/rss/customerreviews/page={page}/id={app_id}/sortby=mostrecent/json"
        
        try:
            resp = requests.get(url)
            if resp.status_code != 200: 
                print(f"Stop : code {resp.status_code} sur page {page}")
                break
            
            data = resp.json()
            entries = data.get('feed', {}).get('entry', [])
            
            if not entries:
                print(f"Stop : plus d'entrées sur page {page}")
                break
            
            for entry in entries:
                # On ignore les entrées qui n'ont pas d'auteur (souvent l'info de l'app)
                if 'author' not in entry: continue
                
                # Conversion de la date
                dt_str = entry['updated']['label']
                review_date = datetime.fromisoformat(dt_str.replace('Z', '+00:00')).date()
                
                if start_date <= review_date <= end_date:
                    all_reviews.append({
                        'source': 'App Store',
                        'date': review_date,
                        'rating': int(entry['im:rating']['label']),
                        'text': entry['content']['label'],
                        'version': entry['im:version']['label']
                    })
                elif review_date < start_date:
                    # Une fois qu'on dépasse la date, on peut tout renvoyer immédiatement
                    return pd.DataFrame(all_reviews)
                    
        except Exception as e:
            print(f"Erreur page {page}: {e}")
            break
            
    return pd.DataFrame(all_reviews)

df_ios = fetch_app_store('6740410176', start, end)
print(df_ios.shape)
df_ios.head(10)

## 4. Consolidation

Nous avons deux DataFrames (`df_gp` et `df_ios`) avec les mêmes colonnes : `source`, `date`, `rating`, `text`, `version`. Il est temps de les fusionner.

In [ ]:
df_final = pd.concat([df_gp, df_ios], ignore_index=True)


# Petit nettoyage final
df_final['date'] = pd.to_datetime(df_final['date'])
df_final = df_final.sort_values('date', ascending=False)

print(f"Total avis récupérés : {len(df_final)}")
print(df_final['source'].value_counts())

# Sauvegarde pour le prochain notebook
# df_final.to_csv('avis_consolides.csv', index=False)
df_final.head(30)

In [ ]:
df_final.to_csv("df_final.csv", index=False)